<h1>Imports </h1>

In [ ]:
import os as os        # for iteration throug directories
import pandas as pd # for Series and DataFrames
import cv2          # for OpenCV 
import numpy as np  # for arrays
import time       # for time calculations
from feature_extraction_try import imgCrawl, getClassLabels #for fetching images
from skimage.feature import hog #calculates HOGs 
from sklearn.cluster import MiniBatchKMeans #for clustering

<h1>HOG computation :</h1>


In this file, we aim to compute a HOG descriptor thas has the same dimension for each image. So we used a bag of word approach : 
<ul>
    <li>Resizing images to a 0mod5 height and width</li>
    <li>Sequencing each image in 5*5 cells</li>
    <li>Computing local histograms on each cell</li>
    <li>Clustering the local histograms</li>
    <li>Binning the local histograms to create our feature</li>
</ul>

<h2>Resizing images : </h2>

We decided to resize images by duplicating the missing pixels on each side (width and height) because it seem to be the less disturbing way for gradient computation. 
Another possible way is to crop each image, but it may delete useful information. 
Returns the resized image as a np.array 

In [ ]:
def reSize(image, CELL_DIMENSION):
  height, width, channels = image.shape
  
  if height%CELL_DIMENSION==0 and width%CELL_DIMENSION==0:
    resizedImage = image
  
  elif width%CELL_DIMENSION==0:
    missingPixels = CELL_DIMENSION-height%CELL_DIMENSION
    resizedImage = cv2.copyMakeBorder(image,0,missingPixels,0,\
                                      0,cv2.BORDER_REPLICATE)
  
  elif height%CELL_DIMENSION==0:
    missingPixels = CELL_DIMENSION-width%CELL_DIMENSION
    resizedImage = cv2.copyMakeBorder(image,0,0,0,missingPixels,\
                                      cv2.BORDER_REPLICATE)
  
  else:
    missingWidthPixels = CELL_DIMENSION-width%CELL_DIMENSION
    missingHeightPixels = CELL_DIMENSION-height%CELL_DIMENSION
    resizedImage = cv2.copyMakeBorder(image,0,missingHeightPixels,0,\
                                      missingWidthPixels,cv2.BORDER_REPLICATE)
  return resizedImage

<h2>Sequencing images : </h2>

Here, we just tranfrom our corpus in $(5*5)$ cells to be able to execute the bag of words approach. 

Returns a (nbImages$*$imageHeight mod 5$*$imageWidth mod 5$*$nbChannels) np.array

In [ ]:
def imageSequencing(npImages, CELL_DIMENSION):
  cells=[]
  
  for k in range(len(npImages)):
    image = cv2.imread(npImages[k][1])
    resizedImage = reSize(image, CELL_DIMENSION)
    height, width, channels = resizedImage.shape
    cells.append(\
      np.array([\
        resizedImage[\
          j*CELL_DIMENSION:j*CELL_DIMENSION+CELL_DIMENSION,\
          i*CELL_DIMENSION:i*CELL_DIMENSION+CELL_DIMENSION] \
        for i in range(width/CELL_DIMENSION) \
        for j in range(height/CELL_DIMENSION)\
      ])\
    )
  return np.array(cells)  

<h2>Compute HOG descriptor on each cell : </h2>

Here we convert our images in grayscale toi be able to apply our localHistogram function (doc : http://scikit-image.org/docs/dev/auto_examples/plot_hog.html)

Then we compute local HOGs on our cells with NB_ORIENTATIONS orientations

In [ ]:
def computeLocalHistograms(cells, NB_ORIENTATIONS, CELL_DIMENSION):
  localHistograms = np.array([\
                      np.array([\
                        hog(cv2.cvtColor( cell, \
                                          cv2.COLOR_BGR2GRAY), \
                                          orientations=NB_ORIENTATIONS, \
                                          pixels_per_cell=(CELL_DIMENSION,\
                                                          CELL_DIMENSION),\
                                          cells_per_block=(1,1)) \
                        for cell in image]) \
                      for image in cells])
  return localHistograms

<h2>Clustering local HOGs : </h2>

Here we search for NB_CLUSTERS clusters in all our localHistograms to be able to bin later our localHistograms.

doc for MiniBatchKMeans : http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html

In [ ]:
def clusterGradients(localHistograms, NB_CLUSTERS, MAXITER):
  sizes = np.array([len(localHistogram) for localHistogram in localHistograms])
  nbImages =  len(localHistograms)
  flattenedHogs = np.array([cell for image in localHistograms for cell in image])
  miniBatchKMeans = MiniBatchKMeans(n_clusters=NB_CLUSTERS, max_iter=MAXITER, \
                    compute_labels=True)
  localHistogramLabels = miniBatchKMeans.fit_predict(flattenedHogs)
  return localHistogramLabels, sizes

<h2>Binning local HOGs : </h2>

Then we just need to bin our local HOGs to avec a constant-sized feature based on HOG to describe our images

In [1]:
def makeHistograms(labels, NB_CLUSTERS, sizes):
  indiceInLabels = 0
  hogs = []
  for image in sizes:
    histogram = np.zeros(NB_CLUSTERS)
    for i in range(image):
      histogram[labels[indiceInLabels+i]] += 1
    hogs.append(histogram)
    indiceInLabels+=i 
  return np.array(hogs)

<h2>Global function</h2>

In [ ]:
def extractHOGFeature(npImages, CELL_DIMENSION, NB_ORIENTATIONS, \
                      NB_CLUSTERS, MAXITER):
  cells = imageSequencing(npImages, CELL_DIMENSION)
  localHistograms = computeLocalHistograms(cells)
  localHistogramLabels, sizes = clusterGradients(localHistograms, \
                                                NB_CLUSTERS, MAXITER)
  hogs = makeHistograms(localHistogramLabels, NB_CLUSTERS, sizes)
  return hogs

<h1>Test zone</h1>

In [ ]:
if __name__ == '__main__':


  start = time.time()
  path ='/home/doob/Dropbox/Marseille/OMIS-Projet/03-jeux-de-donnees/101_ObjectCategories'
  testNpImages = [ [1,'testImage.jpg'] ]
  CELL_DIMENSION = 5
  NB_ORIENTATIONS = 8
  NB_CLUSTERS = 12
  MAXITER = 100

  print "Fetching Images in " + path
  # get dictionary to link classLabels Text to Integers
  sClassLabels = getClassLabels(path)
  # Get all path from all images inclusive classLabel as Integer
  dfImages = imgCrawl(path, sClassLabels)
  npImages = dfImages.values
  extractedTime = time.time()
  print "Extracted images in " + str(extractedTime-start) +'sec'
  print "Sequencing Images ..."
  blocks = imageSequencing(testNpImages, 5)
  sequencedTime = time.time()
  print "Sequenced images in " + str(sequencedTime-extractedTime) +'sec'
  print "Computing gradient on each block ..."
  gradients = computeLocalHistograms(blocks, NB_ORIENTATIONS, CELL_DIMENSION)
  hogedTime = time.time()
  print "Computed gradients in " + str(hogedTime - sequencedTime) + 'sec'
  print "Clustering gradients ..."
  gradientLabels, sizes = clusterGradients(gradients, NB_CLUSTERS, MAXITER)
  clusteredItme = time.time()
  print "Clustered gradients in " + str(hogedTime - sequencedTime) + 'sec'
  print "Computing histograms ..."
  histograms = makeHistograms(gradientLabels, NB_CLUSTERS, sizes)
  end = time.time()
  print "Computed histograms in " + str(int(end - hogedTime)) + 'sec'
  print "Histogram shape : " +str(histograms.shape)
  print "Total time : " + str(end-start) + 'sec'
  #hogs = extractHOGFeature(testNpImages, CELL_DIMENSION, \
  #                         NB_ORIENTATIONS, NB_CLUSTERS, MAXITER)